In [10]:
# import pandas as pd

# data = pd.read_json('categorized-comments.jsonl', lines=True)

ValueError: Unmatched ''"' when when decoding 'string'

In [11]:
# import jsonlines
# # blank list
# data = []
 
# with jsonlines.open('categorized-comments.jsonl') as reader:
#     for obj in reader:
#         data.append(obj)

InvalidLineError: line contains invalid json: Unterminated string starting at: line 1 column 31 (char 30) (line 606476)

In [14]:
import jsonlines

data = []
 
with jsonlines.open('categorized-comments.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        data.append(obj)

In [15]:
len(data)

606475

In [16]:
data[0]

{'cat': 'sports',
 'txt': "Barely better than Gabbert? He was significantly better this year.  He was 16th in passer rating, ahead of guys like Philip Rivers, Carson Palmer, Eli Manning, and Tyrod Taylor.  And that was with the worst surrounding offensive cast in the league.  He is without a doubt better than the career backups you just listed, and although Jimmy G could potentially be better, he's more risky than Kaep is, especially because we'd only get one cheap year with him anyways, and still have to get rid of much more to acquire him."}

In [18]:
len(data[0])

2

In [19]:
df = pd.DataFrame(data)

In [20]:
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [21]:
df.cat.unique()

array(['sports', 'science_and_technology', 'video_games'], dtype=object)

## Twitter Sentiment Analysis - Learn Python for Data Science #2   
https://www.youtube.com/watch?v=o_OZdbCzHUA

In [23]:
from textblob import TextBlob

In [24]:
wiki = TextBlob(df.txt[0])

In [26]:
wiki.tags[:10]

[('Barely', 'RB'),
 ('better', 'JJR'),
 ('than', 'IN'),
 ('Gabbert', 'NNP'),
 ('He', 'PRP'),
 ('was', 'VBD'),
 ('significantly', 'RB'),
 ('better', 'RBR'),
 ('this', 'DT'),
 ('year', 'NN')]

In [27]:
wiki.words

WordList(['Barely', 'better', 'than', 'Gabbert', 'He', 'was', 'significantly', 'better', 'this', 'year', 'He', 'was', '16th', 'in', 'passer', 'rating', 'ahead', 'of', 'guys', 'like', 'Philip', 'Rivers', 'Carson', 'Palmer', 'Eli', 'Manning', 'and', 'Tyrod', 'Taylor', 'And', 'that', 'was', 'with', 'the', 'worst', 'surrounding', 'offensive', 'cast', 'in', 'the', 'league', 'He', 'is', 'without', 'a', 'doubt', 'better', 'than', 'the', 'career', 'backups', 'you', 'just', 'listed', 'and', 'although', 'Jimmy', 'G', 'could', 'potentially', 'be', 'better', 'he', "'s", 'more', 'risky', 'than', 'Kaep', 'is', 'especially', 'because', 'we', "'d", 'only', 'get', 'one', 'cheap', 'year', 'with', 'him', 'anyways', 'and', 'still', 'have', 'to', 'get', 'rid', 'of', 'much', 'more', 'to', 'acquire', 'him'])

Polarity is how positive or negative something is on a scale of -1 to +1:

In [28]:
wiki.sentiment.polarity

0.24

Subjectivity is how much of an opinion it is vs how factual

In [29]:
wiki.sentiment

Sentiment(polarity=0.24, subjectivity=0.67)

## Sentiment Analysis in 4 Minutes  
https://www.youtube.com/watch?v=AJVP96tAWxw

In [36]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier 
# from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import nltk

In [38]:
import re

In [52]:
from nltk.corpus import stopwords

In [48]:
# Remove characters
no_chars = re.sub("[^a-zA-Z]"," ", wiki.string)
no_chars

'Barely better than Gabbert  He was significantly better this year   He was   th in passer rating  ahead of guys like Philip Rivers  Carson Palmer  Eli Manning  and Tyrod Taylor   And that was with the worst surrounding offensive cast in the league   He is without a doubt better than the career backups you just listed  and although Jimmy G could potentially be better  he s more risky than Kaep is  especially because we d only get one cheap year with him anyways  and still have to get rid of much more to acquire him '

In [50]:
# Convert to lowercaase and split
words = no_chars.lower().split()
words[:10]

['barely',
 'better',
 'than',
 'gabbert',
 'he',
 'was',
 'significantly',
 'better',
 'this',
 'year']

In [54]:
# Remove stopwords
stops = set(stopwords.words("english"))
no_stops = [w for w in words if not w in stops]
no_stops[:10]

['barely',
 'better',
 'gabbert',
 'significantly',
 'better',
 'year',
 'th',
 'passer',
 'rating',
 'ahead']

In [55]:
len(words)

93

In [56]:
len(no_stops)

50

Create a bag of words

In [58]:
vectorizer = CountVectorizer(analyzer="word", 
                             tokenizer=None, 
                             preprocessor=None, 
                             stop_words=None, 
                             max_features=5000) #at max 5000 words and their associated frequencies

In [59]:
train_data_features = vectorizer.fit_transform(words)

In [60]:
train_data_features = train_data_features.toarray()

In [61]:
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
train_data_features.shape

(93, 65)

Train the classifier (Random Forest here)  
...this may take a while.

In [63]:
forest = RandomForestClassifier(n_estimators=100) # 100 trees

In [66]:
forest = forest.fit( train_data_features, train["sentiment"]) # not sure here

NameError: name 'train' is not defined

In [67]:
clean_test_reviews = []

In [68]:
test_data_features = vectorizer.transform(clean_test_reviews)

In [69]:
test_data_features = test_data_features.toarray()

Predict the category in the testing data

In [ ]:
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv(os.path.join(os.path.dirname(__file__), 'data', 
                           'Bag_of_Words_model.csv'), index=False, quoting=3) # quoting??